# Kaggle Titanic 생존자 예측하기

캐글의 튜토리얼 중 하나인 타이타닉호 생존자 예측하기입니다.

https://www.kaggle.com/c/titanic/

## Import Graphlab

graphlab 라이브러리를 import합시다.

In [2]:
import graphlab

## 데이터 화일 압축 풀기

먼저 kaggle_titanic.zip 화일을 data 폴더 아래에 업로드합니다. 그런 다음 아래 명령을 실행해서 압축을 풉니다.

In [2]:
!unzip -o ../data/kaggle_titanic.zip -d ../data

Archive:  ../data/kaggle_titanic.zip
   creating: ../data/kaggle_titanic/
  inflating: ../data/kaggle_titanic/test.csv  
  inflating: ../data/kaggle_titanic/gender_submission.csv  
  inflating: ../data/kaggle_titanic/train.csv  


압축이 잘 풀렸는지 꼭 확인합시다.

In [3]:
ls ../data/kaggle_titanic/

gender_submission.csv  test.csv  train.csv


## 데이터 가져오기


먼저 train.csv 화일을 가져 옵시다. 별도의 검증 데이터가 존재하지 않기 때문에 학습 데이터를 가져와서 train_data와 validation_data로 나눠서 사용하고자 합니다. 따라서 전체 학습 데이터는 **full_train_data**라는 변수에 저장합니다.

    graphlab.SFrame(csv화일이름)

In [3]:
full_train_data = graphlab.SFrame('../data/kaggle_titanic/train.csv')

This non-commercial license of GraphLab Create for academic use is assigned to cutty724@hanmail.net and will expire on December 24, 2018.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1520413381.log


Finished parsing file /home/nbuser/library/data/kaggle_titanic/train.csv

Parsing completed. Parsed 100 lines in 1.30375 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,int,int,str,str,float,int,int,str,float,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /home/nbuser/library/data/kaggle_titanic/train.csv

Parsing completed. Parsed 891 lines in 0.568565 secs.

head메소드로 데이터를 확인합시다.

In [4]:
full_train_data.head()

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171
2,1,1,"Cumings, Mrs. JohnBradley (Florence Briggs ...",female,38.0,1,0,PC 17599
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282
4,1,1,"Futrelle, Mrs. JacquesHeath (Lily May Peel) ...",female,35.0,1,0,113803
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450
6,0,3,"Moran, Mr. James",male,None,0,0,330877
7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463
8,0,3,"Palsson, Master. GostaLeonard ...",male,2.0,3,1,349909
9,1,3,"Johnson, Mrs. Oscar W(Elisabeth Vilhelmina ...",female,27.0,0,2,347742
10,1,2,"Nasser, Mrs. Nicholas(Adele Achem) ...",female,14.0,1,0,237736


test.csv에 들어 있는 테스트 데이터를 **test_data**에 저장합시다.

In [5]:
test_data = graphlab.SFrame('../data/kaggle_titanic/test.csv')

Finished parsing file /home/nbuser/library/data/kaggle_titanic/test.csv

Parsing completed. Parsed 100 lines in 0.256549 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,int,str,str,float,int,int,str,float,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /home/nbuser/library/data/kaggle_titanic/test.csv

Parsing completed. Parsed 418 lines in 0.236682 secs.

마찬가지로 head 메소드로 확인합시다.

In [6]:
test_data.head()

PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,,Q
893,3,"Wilkes, Mrs. James (EllenNeeds) ...",female,47.0,1,0,363272,7.0,,S
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,,Q
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,,S
896,3,"Hirvonen, Mrs. Alexander(Helga E Lindqvist) ...",female,22.0,1,1,3101298,12.2875,,S
897,3,"Svensson, Mr. JohanCervin ...",male,14.0,0,0,7538,9.225,,S
898,3,"Connolly, Miss. Kate",female,30.0,0,0,330972,7.6292,,Q
899,2,"Caldwell, Mr. AlbertFrancis ...",male,26.0,1,1,248738,29.0,,S
900,3,"Abrahim, Mrs. Joseph(Sophie Halaut Easu) ...",female,18.0,0,0,2657,7.2292,,C
901,3,"Davies, Mr. John Samuel",male,21.0,2,0,A/4 48871,24.15,,S


그래프 시각화를 위해서 아래 명령을 실행합시다.

In [7]:
graphlab.canvas.set_target('ipynb')

## 데이터 정의

원본은 다음 링크에서 확인하실 수 있습니다.
https://www.kaggle.com/c/titanic/data

<tbody>
<tr><th><b>Variable</b></th><th><b>Definition</b></th><th><b>Key</b></th></tr>
<tr>
<td>survival</td>
<td>Survival</td>
<td>0 = No, 1 = Yes</td>
</tr>
<tr>
<td>pclass</td>
<td>Ticket class</td>
<td>1 = 1st, 2 = 2nd, 3 = 3rd</td>
</tr>
<tr>
<td>sex</td>
<td>Sex</td>
<td></td>
</tr>
<tr>
<td>Age</td>
<td>Age in years</td>
<td></td>
</tr>
<tr>
<td>sibsp</td>
<td># of siblings / spouses aboard the Titanic</td>
<td></td>
</tr>
<tr>
<td>parch</td>
<td># of parents / children aboard the Titanic</td>
<td></td>
</tr>
<tr>
<td>ticket</td>
<td>Ticket number</td>
<td></td>
</tr>
<tr>
<td>fare</td>
<td>Passenger fare</td>
<td></td>
</tr>
<tr>
<td>cabin</td>
<td>Cabin number</td>
<td></td>
</tr>
<tr>
<td>embarked</td>
<td>Port of Embarkation</td>
<td>C = Cherbourg, Q = Queenstown, S = Southampton</td>
</tr>
</tbody>


### 변수 노트

**pclass**: A proxy for socio-economic status (SES)<br>
1st = Upper<br>
2nd = Middle<br>
3rd = Lower

**age**: Age is fractional if less than 1. If the age is estimated, is it in the form of xx.5

**sibsp**: The dataset defines family relations in this way...<br>
Sibling = brother, sister, stepbrother, stepsister<br>
Spouse = husband, wife (mistresses and fiancés were ignored)

**parch**: The dataset defines family relations in this way...<br>
Parent = mother, father<br>
Child = daughter, son, stepdaughter, stepson<br>
Some children travelled only with a nanny, therefore parch=0 for them.<br>

## 데이터 살펴보기


show 메소드로 학습 데이터를 확인합시다.

In [8]:
full_train_data.show()

shape 메소드로 학습데이터의 전체 길이와 컬럼 개수를 확인합시다.

In [9]:
full_train_data.shape

(891, 12)

총 891명, PassengerId와 Survived 제외하면 총 10개의 컬럼 존재

Pclass 	Name 	Sex 	Age 	SibSp 	Parch 	Ticket 	Fare 	Cabin 	Embarked

- Cabin은 비어 있는 데이터가 많음
- Ticket 값은 알 수 없는 값이 많음

## 성별만으로 예측하기


### 성별 데이터 살펴보기

#### 데이터 전처리
- 비어 있는 값 (없는 값)이 있는지 확인하기. 있다면 값 채우기
- 생존율과의 상관관계 확인하기


먼저 성별의 분포를 알아 봅시다.

In [10]:
full_train_data['Sex'].show(view='Categorical')

다행히 비어 있는 값이 없군요.

생존율의 분포도 확인해 봅시다.

In [11]:
full_train_data['Survived'].show(view='Categorical')

성별과 생존율의 상관 관계를 확인해 봅시다.

In [12]:
full_train_data[full_train_data['Survived'] == 0].show(view="Bar Chart", x="Sex")

In [13]:
full_train_data[full_train_data['Survived'] == 1].show(view="Bar Chart", x="Sex")

데이터에는 문제가 없어 보이고 생존율과의 상관관계도 어느 정도 있어 보입니다.

먼저 전체 학습 데이터를 학습 데이터와 검증 데이터로 분리합시다. **train_data, validation_data**라는 변수에 저장합시다.

        SFrame.random_split(비율, seed=시드값)
        
시드값은 0을 사용합시다.

In [14]:
train_data, validation_data = full_train_data.random_split(.8, seed=0)

### 성별 데이터만으로 모델 학습시키기

이제 아래 classifier를 이용해서 모델을 학습시키고 **sex_only_model**에 저장합시다.

    graphlab.logistic_classifier.create(학습데이터, target=타겟컬럼이름, features=피쳐컬럼이름의리스트, validation_set=검증데이터)

In [15]:
sex_only_model = graphlab.logistic_classifier.create(train_data, target='Survived', features=['Sex'], validation_set=validation_data)

Logistic regression:

--------------------------------------------------------

Number of examples          : 703

Number of classes           : 2

Number of feature columns   : 1

...

| Iteration | Passes   | Elapsed Time | Training-accuracy | Validation-accuracy |

+-----------+----------+--------------+-------------------+---------------------+

| 1         | 2        | 1.003285     | 0.800853          | 0.734043            |

| 2         | 3        | 1.006898     | 0.800853          | 0.734043            |

| 3         | 4        | 1.011381     | 0.800853          | 0.734043            |

### 검증데이터를 이용해서 모델 검증하기

show 메소드로 모델의 결과를 살펴 봅시다.
    
    model.show()

In [16]:
sex_only_model.show()

+-----------+----------+--------------+-------------------+---------------------+

SUCCESS: Optimal solution found.

검증 데이터를 이용해서 검증합시다.

    model.evaluate(검증데이터)

In [17]:
sex_only_model.evaluate(validation_data)

{'accuracy': 0.7340425531914894,
 'auc': 0.7204719764011799,
 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      0       |        1        |   24  |
 |      1       |        0        |   26  |
 |      1       |        1        |   49  |
 |      0       |        0        |   89  |
 +--------------+-----------------+-------+
 [4 rows x 3 columns],
 'f1_score': 0.6621621621621622,
 'log_loss': 0.5855304729422249,
 'precision': 0.6712328767123288,
 'recall': 0.6533333333333333,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+-----+-----+----+-----+
 | threshold | fpr | tpr | p  |  n  |
 +-----------+-----+-----+----+-----+
 |    0.0    | 1.0 | 1.0 | 75 | 113 |
 |   1e-05   | 1.0 | 1.0 | 75 | 113 |
 |   2e-05   | 1.0 | 1.0 | 75 | 11

accuracy가 얼마 나왔나요?

위의 코드를 수정해서 accuracy만 출력해 보세요.

In [18]:
sex_only_model.evaluate(validation_data)['accuracy']

0.7340425531914894

#### (고급) 직접 accuracy 구해보기

먼저 학습된 모델을 이용해서 검증 데이터에 대한 예측값을 구합시다. 생존 확률이 0.5 이상인 경우 생존으로 판별하기로 합니다.

참고: output_type을 지정하지 않으면 0과 1로 판별되어 나옵니다. 아래 코드를 이용하면 생존확률의 기준 (0.5)을 높이거나 낮출 수 있습니다. (예. 0.7)

    model.predict(검증데이터, output_type='probability')

In [19]:
sex_only_model.predict(validation_data, output_type='probability')

dtype: float
Rows: 188
[0.17970543072834824, 0.7634083386399846, 0.7634083386399846, 0.17970543072834824, 0.17970543072834824, 0.7634083386399846, 0.7634083386399846, 0.7634083386399846, 0.7634083386399846, 0.17970543072834824, 0.17970543072834824, 0.17970543072834824, 0.17970543072834824, 0.17970543072834824, 0.7634083386399846, 0.7634083386399846, 0.17970543072834824, 0.17970543072834824, 0.7634083386399846, 0.17970543072834824, 0.17970543072834824, 0.17970543072834824, 0.17970543072834824, 0.17970543072834824, 0.7634083386399846, 0.17970543072834824, 0.17970543072834824, 0.17970543072834824, 0.7634083386399846, 0.17970543072834824, 0.17970543072834824, 0.7634083386399846, 0.17970543072834824, 0.7634083386399846, 0.17970543072834824, 0.17970543072834824, 0.17970543072834824, 0.7634083386399846, 0.7634083386399846, 0.17970543072834824, 0.17970543072834824, 0.7634083386399846, 0.7634083386399846, 0.17970543072834824, 0.17970543072834824, 0.17970543072834824, 0.17970543072834824, 0.1797

In [20]:
sex_only_model.predict(validation_data, output_type='probability') >= 0.5

dtype: int
Rows: 188
[0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ... ]

이 결과값을 **predictions**에 저장합시다.

In [21]:
predictions = sex_only_model.predict(validation_data) >= 0.5

이 예측값을 검증데이터의 'Survived' 컬럼과 비교합시다. 즉 모델이 예측한 값이 검증데이터의 관측값과 얼마나 일치하는지 개수를 세는 겁니다. 

참고로 동일한 값을 확인하는 연산자 (operator)는 **==**입니다.

In [22]:
predictions == validation_data['Survived']

dtype: int
Rows: 188
[1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, ... ]

이 결과값을 **corrects**에 저장합시다.

In [23]:
corrects = predictions == validation_data['Survived']

정확도는 아래와 같은 식으로 계산할 수 있습니다. 

    accuracy = 맞은 개수 / 전체 개수
    
여기서 맞은 개수는 어떻게 구할 수 있을까요?

In [24]:
sum(corrects)

138

138개가 나와야 합니다.

이제 정확도(accuracy)를 구해 봅시다.

In [25]:
print(sum(corrects)/float(len(validation_data)))

0.734042553191


0.734042553191가 나와야 합니다.

다음 방법으로 정확도를 간단히 구할 수 있습니다.

In [26]:
sex_only_model.evaluate(validation_data)['accuracy']

0.7340425531914894

### Baseline과 비교하기

이제 baseline을 구해서 우리가 만든 모델의 정확도가 의미 있는 것인지 확인해 봅시다.

random_guessing인 경우 얼마일까요?

In [27]:
validation_data['Sex'].show(view='Categorical')

majority class를 구해 봅시다.

    SFrame.show(view=뷰유형)
    
뷰 유형으로는 'Categorical'을 사용하세요.

어떤 데이터의 어떤 컬럼을 이용해서 majority class를 구해야 할까요?

In [28]:
validation_data['Survived'].show(view='Categorical')

우리가 만든 모델의 예측값이 baseline인 남자의 생존율 60.106%보다 월등히 높다는 것을 알 수 있습니다.

## 더 많은 데이터를 이용한 모델 만들기

이제 추가적인 데이터를 이용해서 모델을 만들어 봅시다.


### 데이터 전처리를 위한 변수 만들기

데이터 처리를 하기 전에 데이터를 새로운 변수에 저장합시다. 이제 수정한 결과값은 아래 두 변수에 저장할 겁니다. 

In [29]:
new_full_train_data = full_train_data
new_test_data = test_data

### Pclass 살펴 보기

먼저 Pclass에 대해서 살펴 봅시다.

In [30]:
full_train_data.head()

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171
2,1,1,"Cumings, Mrs. JohnBradley (Florence Briggs ...",female,38.0,1,0,PC 17599
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282
4,1,1,"Futrelle, Mrs. JacquesHeath (Lily May Peel) ...",female,35.0,1,0,113803
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450
6,0,3,"Moran, Mr. James",male,None,0,0,330877
7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463
8,0,3,"Palsson, Master. GostaLeonard ...",male,2.0,3,1,349909
9,1,3,"Johnson, Mrs. Oscar W(Elisabeth Vilhelmina ...",female,27.0,0,2,347742
10,1,2,"Nasser, Mrs. Nicholas(Adele Achem) ...",female,14.0,1,0,237736


In [31]:
test_data.head()

PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,,Q
893,3,"Wilkes, Mrs. James (EllenNeeds) ...",female,47.0,1,0,363272,7.0,,S
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,,Q
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,,S
896,3,"Hirvonen, Mrs. Alexander(Helga E Lindqvist) ...",female,22.0,1,1,3101298,12.2875,,S
897,3,"Svensson, Mr. JohanCervin ...",male,14.0,0,0,7538,9.225,,S
898,3,"Connolly, Miss. Kate",female,30.0,0,0,330972,7.6292,,Q
899,2,"Caldwell, Mr. AlbertFrancis ...",male,26.0,1,1,248738,29.0,,S
900,3,"Abrahim, Mrs. Joseph(Sophie Halaut Easu) ...",female,18.0,0,0,2657,7.2292,,C
901,3,"Davies, Mr. John Samuel",male,21.0,2,0,A/4 48871,24.15,,S


show 메소드를 이용해서 데이터의 분포를 확인합시다.

    SArray.show(view=뷰유형)

In [32]:
full_train_data['Pclass'].show(view='Categorical')
test_data['Pclass'].show(view='Categorical')

값이 없는 데이터가 있는지 확인합시다.

힌트: None을 이용하면 됩니다.

항상 full_train_data와 test_data 모두를 확인해야 합니다. 왜일까요?

In [33]:
full_train_data[full_train_data['Pclass']==None].show()

In [34]:
test_data[test_data['Pclass']==None].show()

다행히 없군요.

생존율과의 상관 관계를 확인합시다.

In [35]:
full_train_data[full_train_data['Survived'] == 0].show(view="Bar Chart", x="Pclass")

In [36]:
full_train_data[full_train_data['Survived'] == 1].show(view="Bar Chart", x="Pclass")

### Fare 살펴보기

show 메소드로 살펴 봅시다.

In [37]:
full_train_data['Fare'].show()

#### None 찾아서 값 채우기

값이 없는 데이터가 있는지 확인합시다.

In [38]:
full_train_data[full_train_data['Fare']==None]

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked


In [39]:
test_data[test_data['Fare']==None]

PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1044,3,"Storey, Mr. Thomas",male,60.5,0,0,3701,None,,S


test_data에 한명의 데이터가 없다는 것을 알 수 있습니다.

### 비어 있는 값 채우기

비어 있는 값을 어떻게 채우면 좋을까요? 여기에서는 test_data의 평균값을 사용합시다.

먼저 'Fare'의 평균값을 구하세요.

    SArray.mean()

In [40]:
test_data['Fare'].mean()

35.62718848920863

35.627188489208635가 나와야 합니다.

In [41]:
help(graphlab.SFrame.fillna)

Help on method fillna in module graphlab.data_structures.sframe:

fillna(self, column, value) unbound graphlab.data_structures.sframe.SFrame method
    Fill all missing values with a given value in a given column. If the
    ``value`` is not the same type as the values in ``column``, this method
    attempts to convert the value to the original column's type. If this
    fails, an error is raised.
    
    Parameters
    ----------
    column : str
        The name of the column to modify.
    
    value : type convertible to SArray's type
        The value used to replace all missing values.
    
    Returns
    -------
    out : SFrame
        A new SFrame with the specified value in place of missing values.
    
    See Also
    --------
    dropna
    
    Examples
    --------
    >>> sf = graphlab.SFrame({'a':[1, None, None],
    ...                       'b':['13.1', '17.2', None]})
    >>> sf = sf.fillna('a', 0)
    >>> sf
    +---+------+
    | a |  b   |
    +---+------+
    

fillna 메소드를 이용해서 None으로 비어 있는 값을 지정할 수 있습니다.

    SFrame.fillna(column=컬럼이름, value=지정하고자하는값)
    
**주의: 반드시 다시 SFrame에 저장해야 합니다.** 여기서는 **new_test_data**라는 새로운 변수명을 사용합시다.

In [42]:
new_test_data = test_data.fillna(column='Fare', value=test_data['Fare'].mean())

PassengerId가 1044인 승객을 다시 출력해 봅시다.

In [43]:
new_test_data[new_test_data['PassengerId']==1044]

PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1044,3,"Storey, Mr. Thomas",male,60.5,0,0,3701,35.6271884892,,S


Fare에 35.627...로 값이 잘 지정된 것을 확인할 수 있습니다.

생존율과의 상관관계를 확인합시다. 

In [44]:
full_train_data[full_train_data['Survived'] == 0].show(view="Bar Chart", x="Fare")

In [45]:
full_train_data[full_train_data['Survived'] == 1].show(view="Bar Chart", x="Fare")

상관 관계가 있어 보이나요?

In [46]:
#그닥...

### Embarked 살펴보기


show 메소드로 살펴봅시다.

In [47]:
full_train_data['Embarked'].show()

테스트 데이터도 마찬가지로 봅시다.

In [48]:
new_test_data['Embarked'].show()

#### None 찾아서 값 채우기

값이 없는 데이터가 있는지 확인합시다.

In [49]:
full_train_data[full_train_data['Embarked']=='']

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
62,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,
830,1,1,"Stone, Mrs. George Nelson(Martha Evelyn) ...",female,62.0,0,0,113572,80.0,B28,


In [50]:
new_test_data[new_test_data['Embarked']=='']

PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked


full_train_data에 2명의 데이터가 Embarked 값이 없다는 것을 확인할 수 있습니다.
가장 많은 값인 S로 채워봅시다.

먼저 공백을 S로 변환하고 다른 값은 그대로 두는 함수를 만듭니다. 이름은 **fillSforEmbarked**로 합시다.

In [51]:
def fillSforEmbarked(value):
    if value=='':
        return 'S'
    else:
        return value

apply 메소드를 이용해서 Embarked 컬럼에 적용합시다. 이 결과값을 new_full_train_data의 'Embarekd' 컬럼에 저장합시다.

In [57]:
new_full_train_data['Embarked'] = full_train_data['Embarked'].apply(fillSforEmbarked)

PassengerId 62번 승객의 데이터를 찾아서 Embarked가 잘 지정되었는지 확인합시다.

In [59]:
new_full_train_data[new_full_train_data['PassengerId']==62]

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
62,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,S


In [61]:
type(new_full_train_data[new_full_train_data['PassengerId']==62]['Embarked'])

graphlab.data_structures.sarray.SArray

In [62]:
new_full_train_data[new_full_train_data['PassengerId']==62]['Embarked'][0]

'S'

생존율과의 관계를 확인합시다.

In [63]:
new_full_train_data[new_full_train_data['Survived'] == 0].show(view="Bar Chart", x="Embarked")

In [64]:
new_full_train_data[new_full_train_data['Survived'] == 1].show(view="Bar Chart", x="Embarked")

## 4개의 피쳐를 이용해서 모델 학습시키기


위에서 살펴본 4가지 데이터를 이용해서 모델을 학습해 봅시다. 먼저 4가지 데이터 컬럼 이름을 이용해서 **my_features**라는 변수를 지정합시다.

    'Sex', 'Pclass', 'Fare', 'Embarked'

In [65]:
my_features = ['Sex', 'Pclass', 'Fare', 'Embarked']

### 수정한 데이터셋을 나누기

**new_full_train_data**를 **new_train_data**와 **new_validation_data**로 나눕시다.

In [66]:
new_train_data, new_validation_data = new_full_train_data.random_split(.8, seed=0)

먼저 가장 간단한 모델을 이용해서 학습시킵시다.

    graphlab.logistic_classifier.create(학습데이터, target=타겟컬럼이름, features=피쳐컬럼이름의리스트, validation_set=검증데이터)
    
**my_features_model**에 저장합시다.

In [67]:
my_features_model = graphlab.logistic_classifier.create(new_train_data, target='Survived', features=my_features, validation_set=new_validation_data)

Logistic regression:

--------------------------------------------------------

Number of examples          : 703

Number of classes           : 2

Number of feature columns   : 4

...

| 2         | 3        | 0.018796     | 0.795164          | 0.723404            |

| 3         | 4        | 0.033073     | 0.792319          | 0.718085            |

| 4         | 5        | 0.046593     | 0.792319          | 0.718085            |

+-----------+----------+--------------+-------------------+---------------------+

SUCCESS: Optimal solution found.

**my_features_model**의 검증데이터에 대한 정확도를 출력해 봅시다.

In [70]:
my_features_model.evaluate(new_validation_data)['accuracy']

0.7180851063829787

**sex_only_model**의 검증데이터에 대한 정확도를 출력해 봅시다.

In [73]:
sex_only_model.evaluate(new_validation_data)['accuracy']

0.7340425531914894

## 망했어요 vs 아니다

어느 값이 더 큰가요? 모델에 피쳐를 추가하면서 정확도가 나아졌나요?

In [74]:
#아뇨...

## 다른 모델들도 학습시키기

graphlab에서 제공하는 다른 classifier들을 이용해서 학습시켜 봅시다.

- Random Forest
- Decision Tree
- SVM
- Nearest Neighbor

https://turi.com/products/create/docs/graphlab.toolkits.classifier.html

### Random Forest

In [75]:
my_features_model_rf = graphlab.random_forest_classifier.create(new_train_data,
                                                        target='Survived',
                                                        features=my_features,
                                                        validation_set=new_validation_data)

Random forest classifier:

--------------------------------------------------------

Number of examples          : 703

Number of classes           : 2

Number of feature columns   : 4

...

| 3         | 0.018589     | 0.827881          | 0.750000            | 0.423041          | 0.528808            |

| 4         | 0.027235     | 0.837838          | 0.760638            | 0.413484          | 0.523354            |

| 5         | 0.030077     | 0.849218          | 0.750000            | 0.409900          | 0.520365            |

| 6         | 0.036662     | 0.852063          | 0.750000            | 0.405725          | 0.521717            |

+-----------+--------------+-------------------+---------------------+-------------------+---------------------+

In [76]:
my_features_model_rf.evaluate(validation_data)['accuracy']

0.7393617021276596

### Decision Tree

In [77]:
my_features_model_dt = graphlab.decision_tree_classifier.create(new_train_data,
                                                        target='Survived',
                                                        features=my_features,
                                                        validation_set=new_validation_data)

Decision tree classifier:

--------------------------------------------------------

Number of examples          : 703

Number of classes           : 2

Number of feature columns   : 4

...

+-----------+--------------+-------------------+---------------------+-------------------+---------------------+

| Iteration | Elapsed Time | Training-accuracy | Validation-accuracy | Training-log_loss | Validation-log_loss |

+-----------+--------------+-------------------+---------------------+-------------------+---------------------+

| 1         | 0.005852     | 0.853485          | 0.750000            | 0.557577          | 0.598062            |

+-----------+--------------+-------------------+---------------------+-------------------+---------------------+

In [78]:
my_features_model_dt.evaluate(validation_data)['accuracy']

0.75

### SVM

In [79]:
my_features_model_svm = graphlab.svm_classifier.create(new_train_data,
                                                        target='Survived',
                                                        features=my_features,
                                                        validation_set=new_validation_data)

SVM:

--------------------------------------------------------

Number of examples          : 703

Number of classes           : 2

Number of feature columns   : 4

...

| 5         | 9        | 1.000000  | 0.026095     | 0.233286          | 0.303191            |

| 6         | 11       | 1.000000  | 0.028931     | 0.752489          | 0.680851            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

In [80]:
my_features_model_dt.evaluate(validation_data)['accuracy']

0.75

### Nearest Neighbor

In [81]:
my_features_model_nn = graphlab.nearest_neighbor_classifier.create(new_train_data,
                                                                   target='Survived',
                                                                   features=my_features)

Defaulting to brute force instead of ball tree because there are multiple distance components.


Starting brute force nearest neighbors model training.

In [82]:
my_features_model_nn.evaluate(validation_data)['accuracy']

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 188     | 0.142248    | 3.335ms      |

...

+--------------+---------+-------------+--------------+

| 0            | 188     | 0.142248    | 2.744ms      |

| Done         |         | 100         | 227.243ms    |

+--------------+---------+-------------+--------------+

0.7021276595744681

In [83]:
full_train_data[full_train_data['Survived']==1].show(view="Bar Chart", x="Pclass")
full_train_data[full_train_data['Survived']==1].show(view="Bar Chart", x="Sex")
full_train_data[full_train_data['Survived']==1].show(view="Bar Chart", x="Age")
full_train_data[full_train_data['Survived']==1].show(view="Bar Chart", x="SibSp")
full_train_data[full_train_data['Survived']==1].show(view="Bar Chart", x="Parch")
full_train_data[full_train_data['Survived']==1].show(view="Bar Chart", x="Ticket")
full_train_data[full_train_data['Survived']==1].show(view="Bar Chart", x="Fare")
full_train_data[full_train_data['Survived']==1].show(view="Bar Chart", x="Cabin")
full_train_data[full_train_data['Survived']==1].show(view="Bar Chart", x="Embarked")

In [84]:
full_train_data['Age'].show()

## Age 데이터 처리하기

In [85]:
no_age = full_train_data[full_train_data['Age']==None]
no_age['Survived'].show(view='Categorical')
full_train_data['Survived'].show(view='Categorical')
print len(full_train_data)
print len(no_age)

891
177


## Submission

In [86]:
model_for_submission = my_features_model_dt

In [87]:
predictions = model_for_submission.predict(test_data)

In [88]:
submission = graphlab.SFrame({"PassengerId": test_data['PassengerId'], 
                              "Survived": predictions})

In [89]:
submission.export_csv('../data/kaggle_titanic/submission.csv')